In [13]:
!pip install --upgrade --quiet langchain langchain-community langchain-openai "langchain-chroma>=0.1.2"
!pip install --upgrade --quiet pypdf pandas python-dotenv 

You should consider upgrading via the 'C:\Users\Eunha\Documents\Personal work\File-Parser-Llamaparse\myenv\Scripts\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'C:\Users\Eunha\Documents\Personal work\File-Parser-Llamaparse\myenv\Scripts\python.exe -m pip install --upgrade pip' command.


In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
import os
import pandas as pd
from dotenv import load_dotenv
from langchain_core.pydantic_v1 import BaseModel, Field
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(
    model="gpt-4o-mini",
    openai_api_key=OPENAI_API_KEY,
)


## Load 

In [4]:
loader = PyPDFLoader("fake_resume.pdf")
pages = loader.load()
pages


[Document(metadata={'producer': 'Skia/PDF m132', 'creator': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36', 'creationdate': '2025-01-31T00:25:32+00:00', 'title': 'Fake Resume: Sarah Chen - Google Docs', 'moddate': '2025-01-31T00:25:32+00:00', 'source': 'fake_resume.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content="Sarah Chen \n Full Stack Web Developer \n Email:  sarah.chen@email.com \n LinkedIn:  linkedin.com/in/sarahchen \n GitHub:  github.com/sarahcodes \n Portfolio: sarahchen.dev \n Location: San Francisco, CA \n Professional Summary \n Innovative Full Stack Web Developer with 6+ years of experience crafting scalable web  applications and microservices. Specialized in React, Node.js, and cloud architecture. Proven  track record of leading technical teams and implementing CI/CD pipelines that reduced  deployment time by 40%. Passionate about clean code, accessibility, and mentoring junior  de

## Split chunk

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(pages)
chunks


[Document(metadata={'producer': 'Skia/PDF m132', 'creator': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36', 'creationdate': '2025-01-31T00:25:32+00:00', 'title': 'Fake Resume: Sarah Chen - Google Docs', 'moddate': '2025-01-31T00:25:32+00:00', 'source': 'fake_resume.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='Sarah Chen \n Full Stack Web Developer \n Email:  sarah.chen@email.com \n LinkedIn:  linkedin.com/in/sarahchen \n GitHub:  github.com/sarahcodes \n Portfolio: sarahchen.dev \n Location: San Francisco, CA \n Professional Summary \n Innovative Full Stack Web Developer with 6+ years of experience crafting scalable web  applications and microservices. Specialized in React, Node.js, and cloud architecture. Proven  track record of leading technical teams and implementing CI/CD pipelines that reduced  deployment time by 40%. Passionate about clean code, accessibility, and mentoring junior  de

## Creating embed func


In [9]:
def create_text_embedding():
    embeddings = OpenAIEmbeddings()
    return embeddings

embedding_function = create_text_embedding()



## DB

In [10]:
def create_vector_db(chunks, embeddings, persist_directory):
    vectorstore = Chroma.from_documents(
        documents=chunks,
        embedding=embeddings,
        persist_directory=persist_directory,
    )
  
    return vectorstore
vector_store = create_vector_db(chunks, embedding_function, "fake_resume_db")


## Quwery

In [14]:
vectorstore = Chroma(
    persist_directory="fake_resume_db",
    embedding_function=embedding_function
)

## Create the retriever
retriever = vectorstore.as_retriever(search_type="similarity")

#This grabs the most relevant chunks based on the question
relevant_chunks = retriever.invoke("What is the first name in the resume?")




## Template

In [15]:
PROMPT_TEMPLATE = """
You are a helpful assistant that can answer questions about the provided resume.
Use the following pieces of retrieved context to answer the question. If you don't have enough information or don't know the answer, just say "N/A".
{context}
___

Answer the following question based on the context provided.
Question: {question}
"""


context_text = "\n\n \n\n".join([doc.page_content for doc in relevant_chunks])

prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question="What is the first name in the resume?")

print(prompt)
llm.invoke(prompt)


Human: 
You are a helpful assistant that can answer questions about the provided resume.
Use the following pieces of retrieved context to answer the question. If you don't have enough information or don't know the answer, just say "N/A".
Sarah Chen 
 Full Stack Web Developer 
 Email:  sarah.chen@email.com 
 LinkedIn:  linkedin.com/in/sarahchen 
 GitHub:  github.com/sarahcodes 
 Portfolio: sarahchen.dev 
 Location: San Francisco, CA 
 Professional Summary 
 Innovative Full Stack Web Developer with 6+ years of experience crafting scalable web  applications and microservices. Specialized in React, Node.js, and cloud architecture. Proven  track record of leading technical teams and implementing CI/CD pipelines that reduced  deployment time by 40%. Passionate about clean code, accessibility, and mentoring junior  developers. 
 Professional Experience 
 Senior Full Stack Developer 
 TechFlow Solutions  | San Francisco, CA  January 2022  - Present

 

Sarah Chen 
 Full Stack Web Developer 
 E

AIMessage(content='The first name in the resume is Sarah.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 681, 'total_tokens': 691, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BVyfkr9cIFzTPmK9aPKzxcXx1HpP6', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--323d2960-2385-4a00-905f-e3375c71db84-0', usage_metadata={'input_tokens': 681, 'output_tokens': 10, 'total_tokens': 691, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [19]:
def format_document(document):
    return "\n\n".join([doc.page_content for doc in document])

In [17]:
# def format_document(document):
#     return "\n\n".join([doc.page_content for doc in document])



# rag_chain = (
#     {"context": retriever | format_document, "question": RunnablePassthrough()}
#     | prompt_template
#     | llm.with_str
# )

#ignore this 

## Use pydantic to format neatly

In [20]:
class Resume(BaseModel):
    first_name: str = Field(description="The first name of the candidate")
    last_name: str = Field(description="The last name of the candidate")
    email: str = Field(description="The email of the candidate")
    phone: str = Field(description="The phone number of the candidate")
    linkedin: str = Field(description="The linkedin profile of the candidate")
    github: str = Field(description="The github profile of the candidate")
    website: str = Field(description="The website of the candidate")   

    technical_skills: list[str]
    
    


In [21]:
rag_chain = (
    {"context": retriever | format_document, "question": RunnablePassthrough()}
    | prompt_template
    | llm.with_structured_output(Resume, strict=True)
)

structured = rag_chain.invoke("Give me the first name, last name, email, and technical skills of the candidate (frontend and backend)")


c:\Users\Eunha\Documents\Personal work\File-Parser-Llamaparse\myenv\lib\site-packages\langchain_openai\chat_models\base.py:1630: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


## Pandas DF

In [22]:

df = pd.DataFrame([structured.dict()], index=["resume 1"])
df


,first_name,last_name,email,phone,linkedin,github,website,technical_skills
resume 1,Sarah,Chen,sarah.chen@email.com,,linkedin.com/in/sarahchen,github.com/sarahcodes,sarahchen.dev,"[React, Node.js, cloud architecture]"
